# 批踢踢網路爬蟲







## 爬取看板資料

#### 首先使用requests抓取index.html資料，再利用BeautifulSoup解析，取出上一頁的href，再從中獲得總頁數。

In [ ]:
url = 'https://www.ptt.cc/bbs/Gossiping/index.html'
res = requests.get(url,cookies={'over18': '1'})
soup = BeautifulSoup(res.text,'lxml')
tmp = soup.select('.wide')[1]['href']
totalpage = int(tmp[tmp.find('index') + 5 : tmp.find('.html')]) + 1

#### 使用while迴圈從最新一頁往回執行
#### 並且依序抓取內文裡推文的userID與其content，將取出的userID及content加入data建立成dictionary

In [ ]:
while (1):
    for ele in soup.select('.title a'):
        innerRes = requests.get('https://www.ptt.cc' + ele['href'], cookies={'over18':'1'})
        innerSoup = BeautifulSoup(innerRes.text,'html.parser')
        print(innerRes.url)
        for push in innerSoup.select('.push'):
            if push.text == '檔案過大！部分文章無法顯示':
                continue
            userID = push.select('.push-userid')[0].text.strip()
            content = push.select('.push-content')[0].text.replace(':','',1).strip()
            print(userID)
            print(content)
            
            if userID not in data:
                data[userID] = []
                data[userID].append(content)
            else:
                data[userID].append(content)
        
        if totalpage - 1 == 0:
            break
        else:
            totalpage -= 1

abcd1234  
今天天氣真好  
pony666  
我覺得可以  
qaz02589  
我覺得不行  
xxcqc250  
我愛大咪咪  
yang7777  
87  
  
...  

#### 最後輸出成json格式

In [ ]:
json.dump(data, open('Gossiping.json', 'w'))
print(data)

```
{  
    "abcd1234":["今天天氣真好"],  
    "pony666":["我覺得可以","6666666666"],  
    "qaz02589":["我覺得不行","糖糖先記著","先發給他"],  
    "xxcqc250":["我愛大咪咪","雞腿好吃","推"],  
    "yang7777":["87","87878787","8787","87878787878787"],  
    ...  
}  
```


## 加入MySQL資料庫

#### 使用pymysql將資料依照userID、Board、Content欄位加入資料庫中，方便後續查詢。
![table.jpg](https://i.imgur.com/sMfjJIt.jpg)

In [ ]:
boardArray = ['gossiping','baseball','lol','c_chat','stock','tech_job','sex','nba','movie','womentalk','mobilecomm','babymother','japan_travel','e-shopping','car','koreastar','beauty','joke','lifeismoney','marvel','boy-girl','pc_shopping']
db = pymysql.connect(host='140.120.13.242',port=33060,user='root',passwd='udiclab@720',database='yang_ptt',charset='utf8')
num = 0
for bn in boardArray:
    with open(bn + '.json','r') as file:
        data = json.load(file)
        for key,value in data.items():
            cursor = db.cursor()
            cursor.execute('insert into PTT(userID,board,content) values(%s,%s,%s)',(key,bn,json.dumps(value)))
try: db.commit()
except: db.rollback()
db.close()

## 查詢功能

#### 使用MySQL語法將資料從資料庫取出，並提供查詢某用戶在ptt上的留言狀況及在特定看板的留言內容。

In [ ]:
cursor = db.cursor()
cursor.execute('SELECT Board,Content FROM ptt_push as sql1 inner join (SELECT ID FROM ptt_push WHERE userID=%s) as sql2 on sql1.ID = sql2.ID',(UserID))
result = cursor.fetchall()
resultArray = []
if len(result) > 0:
    for ele in result:
        resultArray.append({'Board':ele[0],'Content':ele[1],'Times':len(json.loads(ele[1]))})
    sortedArray = sorted(resultArray, key=operator.itemgetter('Times'), reverse=True)

    for ele in sortedArray:
        print(ele['Board'] + ': ' + str(ele['Times']))

## 查詢demo

![demo](https://i.imgur.com/326Rg7U.jpg)